In [1]:
text='''//label[contains(text(),"Store missing attributes")]
//label[contains(text(),"Store missing attributes")]/i
//label[contains(text(),"Store missing attributes")]/div/span/span[1]
//label[contains(text(),"Store missing attributes")]/div/span/span[2]/i
'''

In [7]:
print text.replace("Store missing attributes","Structural Group Filter").replace("text()",".")

//label[contains(.,"Structural Group Filter")]
//label[contains(.,"Structural Group Filter")]/i
//label[contains(.,"Structural Group Filter")]/div/span/span[1]
//label[contains(.,"Structural Group Filter")]/div/span/span[2]/i



Contents
- [Investigating definition file read](#defread)

In [8]:
import re

In [15]:
def read_conn_file(filepath):
    var_dict={}
    with open(filepath,'r') as conn:
        lines = conn.read()
    roots=re.findall(r'\nroots[\s\S]*?(?=\#)',lines)
    removed_roots=re.sub(r'\nroots[\s\S]*?(?=\#)','',lines)
    reduced_lines=re.sub(r"(#.*\n)","",removed_roots)
    for tup in re.findall(r"\n(\S*)[ |\t]*(.*)",reduced_lines):
        var_dict[tup[0].strip()]=tup[1].strip()
    if roots:
        var_dict['roots']=[x.strip() for x in roots[0].replace('\nroots','').replace('\n\n','').split('\\n\\\n')]
    return {k:v for k,v in var_dict.items() if v!=''}

In [16]:
conn_dict=read_conn_file('ad.tdomain2 - Copy.conn')
display(conn_dict)

{'EnableADDirChgReplication': 'false',
 'IsPagingEnabled': 'true',
 'ScheduledSyncInterv': '0',
 'ScheduledSyncTime': 'Sep 27, 2006 8:14 AM',
 'filter': 'objectclass=*',
 'login': 'john.tyler@tdomain2.qalab01.nextlabs.com',
 'password': '123blue!',
 'port': '389',
 'roots': ['ou=branch1,dc=test,dc=bluejungle,dc=com',
  'ou=branch2,dc=test,dc=bluejungle,dc=com',
  'ou=branch3,dc=test,dc=bluejungle,dc=com'],
 'server': 'w2k12ad2.tdomain2.qalab01.nextlabs.com'}

In [22]:
' '.join(conn_dict['ScheduledSyncTime'].split()[:3])

'Sep 27, 2006'

In [27]:
conn_dict['ScheduledSyncTime'].split()[4]

'AM'

---

### Investigations

- pull variables from pre-defined list

with open('ad.tdomain2 - Copy.conn','r') as conn:
    lines = conn.read()

print lines

lines

server_grp = re.search('server(.*)\n',lines)
server = server_grp.group(1).strip()
print server

port_grp = re.search('port(.*)\n',lines)
port = port_grp.group(1).strip()
print port

login_grp = re.search('login(.*)\n',lines)
login = login_grp.group(1).strip()
print login

re.search('\nroots((.|\n)*)# The LDAP',lines).group(1).replace('\n','').strip()

def pull_vars(text,list_of_vars,roots='yes'):
    var_dict={}
    for var in list_of_vars:
        try:
            grp = re.search(str(var)+'  (.*)\n',text)
            search = grp.group(1).strip()
            var_dict[var]=search
        except:
            pass
    if roots=='yes':
        var_dict['roots']=re.search('\nroots((.|\n)*)# The LDAP',text).group(1).replace('\n','').strip()
    return var_dict

variables = ['server','port','login','password','filter','IsPagingEnabled','EnableADDirChgReplication','ScheduledSyncTime','ScheduledSyncInterv']
print variables

var_dict = pull_vars(lines,variables,roots='no')

for key in var_dict.keys():
    print 'key:{0}, value:{1}'.format(str(key),str(var_dict[key]))

- pull variables based on formatting

lines

roots=re.findall(r'\nroots[\s\S]*?(?=\#)',lines)
print(roots[0])

removed_roots=re.sub(r'\nroots[\s\S]*?(?=\#)','',lines)
print removed_roots

reduced_lines=re.sub(r"(#.*\n)","",removed_roots)
print reduced_lines

re.findall(r"\n(\S*)  (.*)\n",reduced_lines)

var_dict={}
for tup in re.findall(r"\n(\S*)  (.*)\n",reduced_lines):
    var_dict[tup[0].strip()]=tup[1].strip()
var_dict

var_dict

if roots[0]:
    var_dict['roots']=root_list=[x.strip() for x in roots[0].replace('\nroots','').replace('\n\n','').split('\\n\\\n')]

---
### Investigating custom library for checkbox interaction

from robot.libraries.BuiltIn import BuiltIn

def checkbox_interaction(xpath,value):
    se2lib = BuiltIn().get_library_instance('Selenium2Library')
    wd = se2lib._current_browser()
    locator = "document.evaluate('{0}',document,null,XPathResult.FIRST_ORDERED_NODE_TYPE,null).singleNodeValue".format(xpath)
    check = ".checked = {0}".format(value)
    change = ".dispatchEvent(new Event('change'))"
    wd.execute_javascript(locator+check)
    wd.execute_javascript(locator+change)
    return

from Selenium2Library import Selenium2Library as S2L

dir(S2L)

xpath = 'testpath'
value = 'true'
locator = "document.evaluate('{0}',document,null,XPathResult.FIRST_ORDERED_NODE_TYPE,null).singleNodeValue".format(xpath)
check = ".checked = {0}".format(value)
change = ".dispatchEvent(new Event('change'))"

print locator+check
print locator+change

---
### <a id=defread>Investigating definition file read</a>
- text to sections

In [11]:
lines='''# a default SAP enrollment definition file 

enroll.users                        true
enroll.contacts               	    false
enroll.computers                    false
enroll.groups						true
enroll.other              true

#
# Required attributes
#
entry.attributefor.staticid         objectGUID

computer.requirements               objectClass=Computer
user.requirements                   objectClass=User
contact.requirements                objectClass=Contact
other.requirements                  objectClass=Cow

group.requirements                  objectclass=Group
group.attributefor.enumeration      member

structure.requirements				

#
# Attribute mappings

#user.string.accountName            Account Name
#user.string.company                Company 
#user.string.CountryCode            CountryCode
#user.string.countryName	           Country Name
#user.string.department             Department
user.string.displayName            Full Name  
user.string.firstName              First Name
#user.string.isoCountryCode         ISO Country Code 
user.string.lastName               Last Name
user.string.principalName	   User Principal Name
user.string.unixId                 SAPUserId



ldif.filename                       C:/Program Files/Nextlabs/PolicyServer/tools/enrollment/sap_nextlabs_com.ldif
'''

In [12]:
var_dict={}
reduced_lines=re.sub(r"(#.*\n)","",lines)
for tup in re.findall(r"\n([\w|\.]*)[ |\t]*([\S| |\t]*)",reduced_lines):
    var_dict[tup[0].strip()]=tup[1].strip()
def_dict= {k:v for k,v in var_dict.items() if v!=''}
def_dict

{'computer.requirements': 'objectClass=Computer',
 'contact.requirements': 'objectClass=Contact',
 'enroll.computers': 'false',
 'enroll.contacts': 'false',
 'enroll.groups': 'true',
 'enroll.other': 'true',
 'enroll.users': 'true',
 'entry.attributefor.staticid': 'objectGUID',
 'group.attributefor.enumeration': 'member',
 'group.requirements': 'objectclass=Group',
 'ldif.filename': 'C:/Program Files/Nextlabs/PolicyServer/tools/enrollment/sap_nextlabs_com.ldif',
 'other.requirements': 'objectClass=Cow',
 'user.requirements': 'objectClass=User',
 'user.string.displayName': 'Full Name',
 'user.string.firstName': 'First Name',
 'user.string.lastName': 'Last Name',
 'user.string.principalName': 'User Principal Name',
 'user.string.unixId': 'SAPUserId'}

In [44]:
def def_to_dict(filepath):
    var_dict={}
    with open(filepath,'r') as conn:
        lines = conn.read()
    reduced_lines=re.sub(r"(#.*\n)","",lines)
    for tup in re.findall(r"\n([\w|\.]*)[ |\t]*([\S| |\t]*)",reduced_lines):
        var_dict[tup[0].strip()]=tup[1].strip()
    return {k:v for k,v in var_dict.items() if v!=''}

In [51]:
def_dict=def_to_dict('sap_nextlabs_com - Copy.def')
def_dict

{'computer.requirements': 'objectClass=Computer',
 'contact.requirements': 'objectClass=Contact',
 'delete.inactive.group.members': 'false',
 'enroll.computers': 'false',
 'enroll.contacts': 'false',
 'enroll.groups': 'true',
 'enroll.users': 'true',
 'entry.attributefor.staticid': 'objectGUID',
 'group.attributefor.enumeration': 'member',
 'group.requirements': 'objectclass=Group',
 'ldif.filename': 'C:/Program Files/Nextlabs/PolicyServer/tools/enrollment/sap_nextlabs_com.ldif',
 'store.missing.attributes': 'true',
 'user.requirements': 'objectClass=User',
 'user.string.displayName': 'Full Name',
 'user.string.firstName': 'First Name',
 'user.string.lastName': 'Last Name',
 'user.string.principalName': 'User Principal Name',
 'user.string.unixId': 'SAPUserId'}

In [8]:
def conn_to_sub(pattern,main_dict,sub_dict=None):
    del_dict=main_dict.copy()
    if sub_dict==None:
        sub_dict2={}
    else:
        sub_dict2=sub_dict.copy()
    for key in main_dict:
        if re.match(pattern,key)!=None:
            sub_dict2[key]=main_dict[key]
            del_dict.pop(key)
    return sub_dict2, del_dict

In [19]:
to_enroll,def_dict2=conn_to_sub(r'enroll.[\w]*',def_dict)
requirements,def_dict2=conn_to_sub(r'entry.attributefor\.[\w]*',def_dict2)
requirements,def_dict2=conn_to_sub(r'store.missing.attributes',def_dict2,requirements)
requirements,def_dict2=conn_to_sub(r'delete.inactive.group.members',def_dict2,requirements)
if 'enroll.other' in to_enroll.keys():
    to_enroll['enroll.other entities']=to_enroll.pop('enroll.other')
    if 'other.requirements' in def_dict2.keys():
        requirements['other entitie.requirements']=def_dict2.pop('other.requirements')
for key in list(to_enroll.keys()):
    if to_enroll[key]=='false':
        to_enroll.pop(key)
mappings={}
for key in to_enroll.keys():
    entity = key.split('.')[1]
    requirements,def_dict2=conn_to_sub(entity[:-1]+r'\.requirements',def_dict2,requirements)
    requirements,def_dict2=conn_to_sub(entity[:-1]+r'\.attributefor\.[\w]*',def_dict2,requirements)
    mappings[entity],def_dict2=conn_to_sub(entity[:-1]+r'\.[\w]*\.[\w]*',def_dict2)
filename,def_dict2=conn_to_sub(r'ldif.filename',def_dict2)
if 'groups' in mappings.keys():
    if 'structure.requirements' in def_dict2.keys():
        if def_dict2['structure.requirements']!='':
            requirements['structure.requirements']=def_dict2['structure.requirements']

In [20]:
display(requirements)
display (to_enroll)
display (mappings)
display (filename)
display(def_dict2)

{'entry.attributefor.staticid': 'objectGUID',
 'group.attributefor.enumeration': 'member',
 'group.requirements': 'objectclass=Group',
 'other entitie.requirements': 'objectClass=Cow',
 'user.requirements': 'objectClass=User'}

{'enroll.groups': 'true',
 'enroll.other entities': 'true',
 'enroll.users': 'true'}

{'groups': {},
 'other entities': {},
 'users': {'user.string.displayName': 'Full Name',
  'user.string.firstName': 'First Name',
  'user.string.lastName': 'Last Name',
  'user.string.principalName': 'User Principal Name',
  'user.string.unixId': 'SAPUserId'}}

{'ldif.filename': 'C:/Program Files/Nextlabs/PolicyServer/tools/enrollment/sap_nextlabs_com.ldif'}

{'computer.requirements': 'objectClass=Computer',
 'contact.requirements': 'objectClass=Contact'}

In [34]:
def read_def_file(filepath):
    to_enroll,def_dict2=conn_to_sub(r'enroll.[\w]*',def_to_dict(filepath))
    requirements,def_dict2=conn_to_sub(r'entry.attributefor\.[\w]*',def_dict2)
    for key in list(to_enroll.keys()):
        if to_enroll[key]=='false':
            to_enroll.pop(key)
    for key in to_enroll.keys():
        entity = key.split('.')[1]
        mappings={}
        requirements,def_dict2=conn_to_sub(entity[:-1]+r'\.requirements',def_dict2,requirements)
        requirements,def_dict2=conn_to_sub(entity[:-1]+r'\.attributefor\.[\w]*',def_dict2,requirements)
        mappings[entity],def_dict2=conn_to_sub(entity[:-1]+r'\.[\w]*\.[\w]*',def_dict2)
        filename,def_dict2=conn_to_sub(r'ldif.filename',def_dict2)
    return to_enroll, requirements, mappings, filename

In [35]:
read_def_file('sap_nextlabs_com.def')

({'enroll.users': 'true'},
 {'entry.attributefor.staticid': 'objectGUID',
  'user.requirements': 'objectClass=User'},
 {'users': {'user.string.displayName': 'Full Name',
   'user.string.firstName': 'First Name',
   'user.string.lastName': 'Last Name',
   'user.string.principalName': 'User Principal Name',
   'user.string.unixId': 'SAPUserId'}},
 {'ldif.filename': 'C:/Program Files/Nextlabs/PolicyServer/tools/enrollment/sap_nextlabs_com.ldif'})

In [11]:
import json

print (json.dumps(mappings, indent=2))

{
  "users": {
    "user.string.lastName": "Last Name", 
    "user.string.unixId": "SAPUserId", 
    "user.string.displayName": "Full Name", 
    "user.string.firstName": "First Name", 
    "user.string.principalName": "User Principal Name"
  }
}


---
- Sections to dicts

In [14]:
entities_to_enroll_string="""enroll.users                        true
enroll.contacts                     true
enroll.computers                    false
enroll.applications                 true
enroll.groups   	            true"""

In [32]:
entry_filters_string='''user.requirements                   (userPrincipalName=*)
computer.requirements               (dnsHostName=*)
contact.requirements                (objectClass=contact)
group.requirements                  (objectClass=Group)'''
print entry_filters_string

user.requirements                   (userPrincipalName=*)
computer.requirements               (dnsHostName=*)
contact.requirements                (objectClass=contact)
group.requirements                  (objectClass=Group)


In [48]:
def split_list_of_lists(text):
    return [x.split() for x in text.split('\n')]

In [54]:
entities_to_enroll={}
for x in split_list_of_lists(entities_to_enroll_string):
    entities_to_enroll[x[0].split('.')[1]]=x[1]
entities_to_enroll

{'applications': 'true',
 'computers': 'false',
 'contacts': 'true',
 'groups': 'true',
 'users': 'true'}

In [56]:
entry_filters={}
for x in split_list_of_lists(entry_filters_string):
    entry_filters[x[0].split('.')[0]]=x[1].strip('\(\)')    
entry_filters

{'computer': 'dnsHostName=*',
 'contact': 'objectClass=contact',
 'group': 'objectClass=Group',
 'user': 'userPrincipalName=*'}

In [75]:
attributes_string='''computer.string.dnsName                 dnsHostName
user.displayName    name
user.principalName    userPrincipalName
computer.cs-string.windowsSid           objectSid'''
print attributes_string

computer.string.dnsName                 dnsHostName
user.displayName    name
user.principalName    userPrincipalName
computer.cs-string.windowsSid           objectSid


In [76]:
attributes={}
for x in split_list_of_lists(attributes_string):
    entity=x[0].split('.')[0]
    cc_att=x[0].split('.')[-1]
    dir_att=x[1]
    if entity not in attributes:
        attributes[entity]={}
    if cc_att not in attributes[entity]:
        attributes[entity][cc_att]={}
    attributes[entity][cc_att]=dir_att
attributes    

{'computer': {'dnsName': 'dnsHostName', 'windowsSid': 'objectSid'},
 'user': {'displayName': 'name', 'principalName': 'userPrincipalName'}}

In [80]:
import json

print (json.dumps(attributes, indent=2))

{
  "computer": {
    "dnsName": "dnsHostName", 
    "windowsSid": "objectSid"
  }, 
  "user": {
    "displayName": "name", 
    "principalName": "userPrincipalName"
  }
}
